In [ ]:
import datetime
from dataclasses import dataclass
from dateutil import parser as date_parser

import numpy as np

import pandas as pd
from pathlib import Path

In [ ]:
data_dir = Path(".").parent.resolve().parent / "data"
assert data_dir.is_dir()

In [ ]:
df = pd.read_csv(data_dir / "participants_66360376183.csv")

In [ ]:
df.rename(
    columns={"Name (Original Name)": "name", "Join Time": "start", "Leave Time": "end"},
    inplace=True,
)
df["start"] = pd.to_datetime(df["start"])
df["end"] = pd.to_datetime(df["end"])

In [ ]:
df

In [ ]:
names = df.name.unique()

In [ ]:
@dataclass
class Session:
    def __init__(self, name, start, end):
        self.name = name
        self.start = date_parser.parse(start)
        self.end = date_parser.parse(end)

    @property
    def duration(self):
        return self.end - self.start

In [ ]:
np.maximum(df.start.to_numpy(), pd.to_datetime(["2020-10-01 00:00:00"]))

In [ ]:
np.array([date_parser.parse("2020-10-01 00:00:00")] * len(df)).dtype

In [ ]:
df.start.to_numpy() > df.start.to_numpy()

In [ ]:
sessions[0].start

In [ ]:
def get_n_participants_of_session(
    df: pd.DataFrame, session: Session, min_time_frac=0.3
) -> int:
    _a = np.minimum(df["end"], pd.to_datetime([session.end] * len(df)))
    _b = np.maximum(df["start"], pd.to_datetime([session.start] * len(df)))
    df["tmp_overlap_min"] = (_a - _b).dt.total_seconds() / 60
    min_time_min = (session.duration * min_time_frac).total_seconds() / 60
    return len(df.query(f"tmp_overlap_min > {min_time_min}").name.unique())

In [ ]:
sessions = [
    Session("bash", "2022-09-28 10:00", "2022-09-28 12:30"),
    Session("python 1", "2022-09-28 13:45", "2022-09-28 17:00"),
    Session("git", "2022-09-29 10:00", "2022-09-28 13:00"),
    Session("python 2", "2022-09-29 13:45", "2022-09-28 17:00"),
]

In [ ]:
for s in sessions:
    print(f"{s.name}: {get_n_participants_of_session(df, s)}")